<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 初始

每台伺服器在嘗試取得鎖定時會先將自己的識別寫入到 server_id.txt 檔案中，其他伺服器在等待時可以知道哪個伺服器在使用 collection.py 檔案。 這樣，即使其他伺服器想要執行 collection.py 文件，也能夠知道目前是哪個伺服器在運作。

In [ ]:
from multiprocessing import Pool
import os
import time

# 定义锁文件路径和服务器标识文件路径
LOCK_FILE = "test_lock"
SERVER_ID_FILE = "server_id.txt"

def acquire_lock(server_id):
    """获取锁，并记录服务器标识"""
    while os.path.exists(LOCK_FILE):
        print(f"Another server (ID: {server_id}) is running test.py. Waiting for 5 seconds...")
        time.sleep(5)
    # 创建锁文件
    try:
        open(LOCK_FILE, "w").close()
        with open(SERVER_ID_FILE, "w") as f:
            f.write(str(server_id))
    except Exception as e:
        print("Failed to acquire lock:", str(e))
        # 在这里处理获取锁失败的情况

def release_lock():
    """释放锁"""
    try:
        if os.path.exists(LOCK_FILE):
            os.remove(LOCK_FILE)
        if os.path.exists(SERVER_ID_FILE):
            os.remove(SERVER_ID_FILE)
    except Exception as e:
        print("Failed to release lock:", str(e))
        # 在这里处理释放锁失败的情况

def collection(option):
    """执行 collection 函数，并在执行前后处理锁"""
    try:
        # 获取服务器标识
        with open(SERVER_ID_FILE) as f:
            server_id = f.read()
        acquire_lock(server_id)
        # 在这里执行 collection 函数的内容
        print(f"Process {option} (Server {server_id}) is running collection...")
        time.sleep(10)  # 模拟 collection 函数的执行时间
    except Exception as e:
        print("Error in collection:", str(e))
        # 在这里处理 collection 函数执行过程中的异常
    finally:
        release_lock()

if __name__ == "__main__":
    # 假设每个服务器有一个唯一的ID，可以根据需要自行获取
    server_id = "Server1"  # 你需要替换为服务器的实际标识

    pool = Pool(processes=3)

    for i, mode in enumerate(['today', 'recent', 'past']):
        pool.apply_async(collection, args=(mode,))
    pool.close()
    pool.join()

## second

在 if __name__ == "__main__": 中调用，则只会在主进程中执行一次锁的获取和释放操作。而如果在 collection 函数中调用，则每个子进程在执行时都会尝试获取锁，并在执行结束后释放锁。

这里是在 if __name__ == "__main__": 中调用的示例代码：

由於是 multiprocess，最好在主程式中使用 acquire_lock(server_id)，因為這樣可以確保在進程池中的每個進程啟動之前都取得了鎖定。 在每個子程序的 collection 函數中再次取得鎖也可以，但可能會導致多次嘗試取得鎖，從而增加不必要的等待時間。 因此，建議在主程式中取得鎖。

In [ ]:
from multiprocessing import Pool
import os
import time

# 定义锁文件路径和服务器标识文件路径
LOCK_FILE = "test_lock"
SERVER_ID_FILE = "server_id.txt"

def acquire_lock(server_id):
    """获取锁，并记录服务器标识"""
    while True:
        if os.path.exists(LOCK_FILE):
            # 检查锁文件中记录的服务器标识
            with open(SERVER_ID_FILE) as f:
                locked_server_id = f.read().strip()
            if locked_server_id == server_id:
                # 锁文件是当前服务器创建的，但是进程意外终止导致锁文件未被释放
                print(f"Lock file exists but not released by Server {server_id}. Attempting to remove and re-acquire...")
                try:
                    os.remove(LOCK_FILE)
                except Exception as e:
                    print("Failed to remove lock file:", str(e))
                    # 在这里处理无法删除锁文件的情况
            else:
                print(f"Another server (ID: {locked_server_id}) is running test.py. Waiting for 5 seconds...")
                time.sleep(5)
        else:
            # 创建锁文件并记录服务器标识
            try:
                open(LOCK_FILE, "w").close()
                with open(SERVER_ID_FILE, "w") as f:
                    f.write(str(server_id))
                break
            except Exception as e:
                print("Failed to acquire lock:", str(e))
                # 在这里处理获取锁失败的情况

def release_lock(server_id):
    """释放锁"""
    try:
        if os.path.exists(LOCK_FILE):
            os.remove(LOCK_FILE)
        if os.path.exists(SERVER_ID_FILE):
            os.remove(SERVER_ID_FILE)
    except Exception as e:
        print("Failed to release lock:", str(e))
        # 在这里处理释放锁失败的情况

def collection(option):
    """执行 collection 函数，并在执行前后处理锁"""
    try:
        # 获取服务器标识
        with open(SERVER_ID_FILE) as f:
            server_id = f.read().strip()
        # 在这里执行 collection 函数的内容
        print(f"Process {option} (Server {server_id}) is running collection...")
        time.sleep(10)  # 模拟 collection 函数的执行时间
    except Exception as e:
        print("Error in collection:", str(e))
        # 在这里处理 collection 函数执行过程中的异常

if __name__ == "__main__":
    # 假设每个服务器有一个唯一的ID，可以根据需要自行获取
    server_id = "Server1"  # 你需要替换为服务器的实际标识

    # 尝试获取锁并执行 collection.py
    acquire_lock(server_id)
    try:
        pool = Pool(processes=3)
        for i, mode in enumerate(['today', 'recent', 'past']):
            pool.apply_async(collection, args=(mode,))
        pool.close()
        pool.join()
    finally:
        release_lock(server_id)


Q :如果正在使用服务器66运行该代码，而服务器76尝试运行相同的代码，结果会如下：

A :
1.   服务器76尝试获取锁文件，发现已经存在锁文件。
2.   服务器76检查锁文件中记录的服务器标识，发现标识为服务器66。
3.   服务器76显示等待5秒钟后，再次尝试获取锁文件。
4.   服务器76循环等待，直到服务器66的进程完成并释放了锁文件为止。
5.   一旦服务器66的进程完成并释放了锁文件，服务器76的进程就能够获取到锁文件，并且开始执行任务。

因此，在这种情况下，服务器76会等待服务器66的进程执行完成后才能执行任务。



Q :  這樣的話如果有正在使用66運行該code，斷網後，66重新運行，會返回什麼? 該程式能正常運作嗎?

A :

如果在運行該程式碼的過程中斷網，然後重新運行，情況會有所不同：

(1)如果運行過程中斷網，然後重新運行程序，66重新運行，因為在重新運行時沒有其他伺服器持有鎖文件，因此可以獲取鎖，並成功執行 collection.py。 這是因為在重新運行時，程式會嘗試取得鎖，並成功獲取，然後正常執行。

(2)如果運行過程中斷網，然後重新運行程序，但在66重新運行之前另一台伺服器76已經獲取了鎖並正在運行collection.py，那麼66在嘗試獲取鎖時會發現鎖文件已經存在，並且伺服器標識 文件中記錄的伺服器ID 不符合目前伺服器ID。 這時，66會等待一段時間後再嘗試重新取得鎖。 當76執行完成並釋放鎖定後，66將能夠取得鎖定並正常執行 collection.py。


## third : 多進程造成lock重複刪除創建

創建不同mode下的lock文件

In [ ]:
from multiprocessing import Pool
import os
import time

# 定义锁文件的文件夹路径和服务器标识文件路径
LOCK_FOLDER = "lock_files"
SERVER_ID_FILE = "server_id.txt"

def acquire_lock(server_id, mode):
    """获取锁，并记录服务器标识"""
    lock_file = os.path.join(LOCK_FOLDER, f"{mode}_lock")
    while True:
        if os.path.exists(lock_file):
            # 检查锁文件中记录的服务器标识
            with open(SERVER_ID_FILE) as f:
                locked_server_id = f.read().strip()
            if locked_server_id == server_id:
                # 锁文件是当前服务器创建的，但是进程意外终止导致锁文件未被释放
                print(f"Lock file exists but not released by Server {server_id} for mode {mode}. Attempting to remove and re-acquire...")
                try:
                    os.remove(lock_file)
                except Exception as e:
                    print("Failed to remove lock file:", str(e))
                    # 在这里处理无法删除锁文件的情况
            else:
                print(f"Another server (ID: {locked_server_id}) is running test.py for mode {mode}. Waiting for 5 seconds...")
                time.sleep(5)
        else:
            # 创建锁文件并记录服务器标识
            try:
                os.makedirs(LOCK_FOLDER, exist_ok=True)
                open(lock_file, "w").close()
                with open(SERVER_ID_FILE, "w") as f:
                    f.write(str(server_id))
                break
            except Exception as e:
                print("Failed to acquire lock:", str(e))
                # 在这里处理获取锁失败的情况

def release_lock(server_id, mode):
    """释放锁"""
    try:
        lock_file = os.path.join(LOCK_FOLDER, f"{mode}_lock")
        if os.path.exists(lock_file):
            os.remove(lock_file)
        if os.path.exists(SERVER_ID_FILE):
            os.remove(SERVER_ID_FILE)
    except Exception as e:
        print("Failed to release lock:", str(e))
        # 在这里处理释放锁失败的情况

def collection(option):
    """执行 collection 函数，并在执行前后处理锁"""
    try:
        mode, index = option
        # 获取服务器标识
        with open(SERVER_ID_FILE) as f:
            server_id = f.read().strip()
        # 在这里执行 collection 函数的内容
        print(f"Process {mode} (Server {server_id}) is running collection...")
        time.sleep(10)  # 模拟 collection 函数的执行时间
    except Exception as e:
        print("Error in collection:", str(e))
        # 在这里处理 collection 函数执行过程中的异常

if __name__ == "__main__":
    # 假设每个服务器有一个唯一的ID，可以根据需要自行获取
    server_id = "Server1"  # 你需要替换为服务器的实际标识

    # 尝试获取锁并执行 collection.py
    for mode in ['today', 'recent', 'past']:
        acquire_lock(server_id, mode)
    try:
        pool = Pool(processes=3)
        modes = [(mode, i) for i, mode in enumerate(['today', 'recent', 'past'])]
        pool.map(collection, modes)
    finally:
        for mode in ['today', 'recent', 'past']:
            release_lock(server_id, mode)


* 參考命名方式

In [ ]:
from multiprocessing import Pool
import os
import time

# 定义锁文件基本路径和服务器标识文件基本路径
LOCK_FILE_BASE = "test_lock_"
SERVER_ID_FILE_BASE = "server_id_"

def acquire_lock(mode):
    """获取锁，并记录服务器标识"""
    lock_file = LOCK_FILE_BASE + mode
    server_id_file = SERVER_ID_FILE_BASE + mode
    while True:
        if os.path.exists(lock_file):
            # 检查锁文件中记录的服务器标识
            with open(server_id_file) as f:
                locked_server_id = f.read().strip()
            print(f"Another process for mode {mode} (Server {locked_server_id}) is running test.py. Waiting for 5 seconds...")
            time.sleep(5)
        else:
            # 创建锁文件并记录服务器标识
            try:
                open(lock_file, "w").close()
                with open(server_id_file, "w") as f:
                    f.write(str(os.getpid()))
                break
            except Exception as e:
                print(f"Failed to acquire lock for mode {mode}:", str(e))
                # 在这里处理获取锁失败的情况

def release_lock(mode):
    """释放锁"""
    lock_file = LOCK_FILE_BASE + mode
    server_id_file = SERVER_ID_FILE_BASE + mode
    try:
        if os.path.exists(lock_file):
            os.remove(lock_file)
        if os.path.exists(server_id_file):
            os.remove(server_id_file)
    except Exception as e:
        print(f"Failed to release lock for mode {mode}:", str(e))
        # 在这里处理释放锁失败的情况

def collection(mode):
    """执行 collection 函数，并在执行前后处理锁"""
    try:
        # 获取服务器标识
        server_id_file = SERVER_ID_FILE_BASE + mode
        with open(server_id_file) as f:
            server_id = f.read().strip()
        # 在这里执行 collection 函数的内容
        print(f"Process for mode {mode} (Server {server_id}) is running collection...")
        time.sleep(10)  # 模拟 collection 函数的执行时间
    except Exception as e:
        print(f"Error in collection for mode {mode}:", str(e))
        # 在这里处理 collection 函数执行过程中的异常

if __name__ == "__main__":
    # 假设每个服务器有一个唯一的ID，可以根据需要自行获取
    server_id = "Server1"  # 你需要替换为服务器的实际标识

    # 尝试获取锁并执行 collection.py
    try:
        pool = Pool(processes=3)
        for mode in ['today', 'recent', 'past']:
            acquire_lock(mode)
            pool.apply_async(collection, args=(mode,))
        pool.close()
        pool.join()
    finally:
        # 释放所有模式的锁
        for mode in ['today', 'recent', 'past']:
            release_lock(mode)
